# Interact to Redis db

In [2]:
import sys
import os
from langchain.schema import Document
from typing import Dict, List
import json
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.redis import Redis

sys.path.append('../')
from botcore.setup import get_openai_embeddings, load_my_env


class RedisVectorDB:

    def __init__(self):
        load_my_env()
        self.embeddings = get_openai_embeddings()
        self.url = os.getenv("REDIS_CLOUD")
        self.count = 0
        self.redis = None
        self.limit = 0.2
        print("Vector DB is ready")
        
    
    def json_to_doc(self, data: Dict) -> Document:
        """
            data = {"title": str, "features": [], "post_id": str, ...}
        """
        feats = ", ".join([i for i in data['features']])
        txt = f"{data['title']}. {feats}"
        return Document(page_content=txt, metadata={"post_id": data['post_id'], "user_id": data['user_id']})

    def add_new_wanted(self, data: Dict):
        doc = self.json_to_doc(data)
        return self.add_new_doc(doc, 'wanted')

    def add_new_stock(self, data: Dict):
        doc = self.json_to_doc(data)
        return self.add_new_doc(doc, 'stock')
    
    def add_new_doc(self, doc: Document, index_name: str):
        try:
            if self.redis is None:
                self.redis = Redis.from_documents([doc], self.embeddings, redis_url=self.url, index_name=index_name)
            else: 
                self.redis.add_documents([doc])
            return True
        except:
            print("An exception occurred when adding document") 
            return False
            
    def search_wanted(self, query: str):
        if self.redis is None:
            print("Redis is not initialized. Please add a document first")
            return False
        try:
            results = self.redis.similarity_search_limit_score(query, score_threshold=self.limit)
            return results
        except:
            print("Error occurred when finding documents")
            return False

In [3]:
data = [{
  "post_id": "001",
  "user_id": "A001",
  "date": "3-7-2023",
  "title": "I need a used laptop for my kid",
  "features": [
    "The laptop should have 4 GB RAM",
    "The laptop's battery should be durable",
    "The laptop should function normally"
  ]
},{
  "post_id": "002",
  "user_id": "A002",
  "date": "20-7-2023",
  "title": "I need a used rice cooker",
  "features": [
    "The rice cooker should not be damaged too much",
    "The rice cooker should function normally"
  ]
},{
  "post_id": "003",
  "user_id": "A001",
  "date": "20-7-2023",
  "title": "We are in need of small children shirts",
  "features": [
    "The shirts should be for kid from 5 to 10 years old.",
    "The shirts should be intact, and there are no damages.",
    "The shirts should be clean."
  ]
},{
  "post_id": "004",
  "user_id": "A003",
  "date": "3-8-2023",
  "title": "I need a used laptop for learning",
  "features": [
    "The laptop should have 8 GB RAM",
    "The laptop should function normally"
  ]
}]

redis_db = RedisVectorDB()
[redis_db.add_new_wanted(a) for a in data]

OPEN AI Embedding ready
Vector DB is ready


[True, True, True, True]

In [6]:
dir(redis_db.redis)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_create_index',
 '_prepare_query',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_texts',
 'afrom_documents',
 'afrom_texts',
 'amax_marginal_relevance_search',
 'amax_marginal_relevance_search_by_vector',
 'as_retriever',
 'asearch',
 'asimilarity_search',
 'asimilarity_search_by_vector',
 'asimilarity_search_with_relevance_scores',
 'client',
 'content_key',
 'drop_index',
 'embedding_function',
 'from_documents',
 'from_existing_index',
 'from_texts',
 'from_texts_return_keys',
 'index_name',

In [3]:
b = redis_db.search_wanted("I have an old Asus laptop. It has 8 GB RAM.")
b

[Document(page_content='I need a used laptop for learning. The laptop should have 8 GB RAM, The laptop should function normally', metadata={'post_id': '004', 'user_id': 'A003'}),
 Document(page_content="I need a used laptop for my kid. The laptop should have 4 GB RAM, The laptop's battery should be durable, The laptop should function normally", metadata={'post_id': '001', 'user_id': 'A001'})]

In [23]:
dir(redis_db.redis)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_create_index',
 '_prepare_query',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_texts',
 'afrom_documents',
 'afrom_texts',
 'amax_marginal_relevance_search',
 'amax_marginal_relevance_search_by_vector',
 'as_retriever',
 'asearch',
 'asimilarity_search',
 'asimilarity_search_by_vector',
 'asimilarity_search_with_relevance_scores',
 'client',
 'content_key',
 'drop_index',
 'embedding_function',
 'from_documents',
 'from_existing_index',
 'from_texts',
 'from_texts_return_keys',
 'index_name',

# run

In [3]:
import sys
import os
sys.path.append(f'../')
from botcore.bot_redis import RedisVectorDB
from botcore.test_data import TEST_WANTED_DATA 

redis_db = RedisVectorDB()
data = TEST_WANTED_DATA

[redis_db.add_new_wanted(a) for a in data]

OPEN AI Embedding ready
Vector DB is ready


[True, True, True, True]

In [4]:
data

[{'post_id': '001',
  'user_id': 'A001',
  'date': '3-7-2023',
  'title': 'I need a used laptop for my kid',
  'features': ['The laptop should have 4 GB RAM',
   "The laptop's battery should be durable",
   'The laptop should function normally']},
 {'post_id': '002',
  'user_id': 'A002',
  'date': '20-7-2023',
  'title': 'I need a used rice cooker',
  'features': ['The rice cooker should not be damaged too much',
   'The rice cooker should function normally']},
 {'post_id': '003',
  'user_id': 'A001',
  'date': '20-7-2023',
  'title': 'We are in need of small children shirts',
  'features': ['The shirts should be for kid from 5 to 10 years old.',
   'The shirts should be intact, and there are no damages.',
   'The shirts should be clean.']},
 {'post_id': '004',
  'user_id': 'A003',
  'date': '3-8-2023',
  'title': 'I need a used laptop for learning',
  'features': ['The laptop should have 8 GB RAM',
   'The laptop should function normally']}]

In [5]:
# after user input

data = {"title": "I have an old phone", "features": ["My laptop has 4 GB RAM", "Is it function well? Well", "Does it have warranty: Yes"]}

b = redis_db.search_wanted(data)
print(b)

[Document(page_content="I need a used laptop for my kid. The laptop should have 4 GB RAM, The laptop's battery should be durable, The laptop should function normally", metadata={'type': 'wanted'}), Document(page_content='I need a used laptop for learning. The laptop should have 8 GB RAM, The laptop should function normally', metadata={'type': 'wanted'})]


In [6]:
type(b)

list

# find

In [1]:
import sys
import os
sys.path.append(f'../')
from botcore.bot_redis import RedisDB
from botcore.test_data import TEST_WANTED_DATA 

redis_db = RedisDB()
data = TEST_WANTED_DATA

[redis_db.add_new_wanted(a) for a in data]



OPEN AI Embedding ready


[True, True, True, True]

In [4]:

from langchain.vectorstores.redis import Redis

data = {"title": "I have an old phone", "product": "laptop", "features": ["My laptop has 4 GB RAM", "Is it function well? Well"]}
index_name = f'wanted {data["product"]}'
redis = Redis(redis_url = redis_db.url, index_name = 'doc:wanted_laptop',\
                    embedding_function=redis_db.embeddings.embed_query)
doc = redis_db.json_to_doc(data, {"type": index_name})
query = doc.page_content
print(query)
print(index_name)

I have an old phone. My laptop has 4 GB RAM, Is it function well? Well
wanted laptop


In [5]:
res = redis.similarity_search_limit_score(query)

ResponseError: doc:wanted_laptop: no such index

In [ ]:

redis = Redis(redis_url=redis_db.url, )